In [1]:
from demo  import jdt_db
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import time


In [2]:
### 本月时间计算
sql_time_min ="'2022-07-01 00:00:00'"
sql_time_max ="'2022-07-28 00:00:00'"
### 上个月时间
sql_old_time_min ="'2022-06-01 00:00:00'"
sql_old_time_max ="'2022-07-01 00:00:00'"
### 去年月时间
sql_old_time_min_1 ="'2021-07-01 00:00:00'"
sql_old_time_max_1 ="'2021-07-28 00:00:00'"
### 前个月时间
sql_old_time_min_2 ="'2022-05-01 00:00:00'"
sql_old_time_max_2 ="'2022-06-01 00:00:00'"
# 时间戳
list_time1 = 1654012800
list_time2 = 1656604800
list_time3 = 1658937600
# 90天时间
day_90_sql_time_1= "'2022-05-01 00:00:00'"
day_90_sql_time_2= "'2022-04-01 00:00:00'"


# 3.用户留存

### 3.1数据导入

In [3]:
user_retained_sql_1 = "    SELECT    a.id,    a.user_province,    DATE_FORMAT( FROM_UNIXTIME( jointime, '%Y-%m-%d %H:%i:%s' ), '%Y-%m-%d' ) AS '日期',    FROM_UNIXTIME( jointime, '%Y-%m-%d %H:%i:%s' ) AS '时间',    a.age,    a.gender,    a.group_id,    b.created_at,    DATE_FORMAT( b.created_at, '%Y-%m-%d' ) AS 'time'     FROM    jdt_user AS a    LEFT JOIN jdt_user_logon_in AS b ON a.id = b.user_id     WHERE    FROM_UNIXTIME( jointime, '%Y-%m-%d %H:%i:%s' ) >= " + sql_time_min + "     AND FROM_UNIXTIME( jointime, '%Y-%m-%d %H:%i:%s' ) < " + sql_time_max + "     AND b.created_at >= " + sql_time_min + "    AND b.created_at < " + sql_time_max
user_retained_1 = jdt_db(user_retained_sql_1)
user_retained_sql_2 = "    SELECT    a.id,    a.user_province,    DATE_FORMAT( FROM_UNIXTIME( jointime, '%Y-%m-%d %H:%i:%s' ), '%Y-%m-%d' ) AS '日期',    FROM_UNIXTIME( jointime, '%Y-%m-%d %H:%i:%s' ) AS '时间',    a.age,    a.gender,    a.group_id,    b.created_at,    DATE_FORMAT( b.created_at, '%Y-%m-%d' ) AS 'time'     FROM    jdt_user AS a    LEFT JOIN jdt_user_logon_in AS b ON a.id = b.user_id     WHERE    FROM_UNIXTIME( jointime, '%Y-%m-%d %H:%i:%s' ) >= " + sql_old_time_min + "     AND FROM_UNIXTIME( jointime, '%Y-%m-%d %H:%i:%s' ) < " + sql_old_time_max + "     AND b.created_at >= " + sql_old_time_min + "    AND b.created_at < " + sql_old_time_max 
user_retained_2 = jdt_db(user_retained_sql_2)

D:\anaconda\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


4.775823354721069


D:\anaconda\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


2.818427801132202


In [4]:
user_retained_1

,id,user_province,日期,时间,age,gender,group_id,created_at,time
0,2140809,None,2022-07-01,2022-07-01 00:04:08,0,0,1,2022-07-01 00:04:08,2022-07-01
1,2140810,None,2022-07-01,2022-07-01 00:04:13,0,0,1,2022-07-01 00:04:13,2022-07-01
2,2140811,广西壮族自治区,2022-07-01,2022-07-01 00:12:57,36,2,2,2022-07-01 00:12:57,2022-07-01
3,2140811,广西壮族自治区,2022-07-01,2022-07-01 00:12:57,36,2,2,2022-07-01 00:13:29,2022-07-01
4,2140811,广西壮族自治区,2022-07-01,2022-07-01 00:12:57,36,2,2,2022-07-01 00:13:49,2022-07-01
...,...,...,...,...,...,...,...,...,...
85388,2203651,None,2022-07-27,2022-07-27 23:52:34,0,0,1,2022-07-27 23:52:34,2022-07-27
85389,2203652,河北省,2022-07-27,2022-07-27 23:52:58,0,0,1,2022-07-27 23:52:58,2022-07-27
85390,2203652,河北省,2022-07-27,2022-07-27 23:52:58,0,0,1,2022-07-27 23:53:26,2022-07-27
85391,2203653,None,2022-07-27,2022-07-27 23:59:25,0,0,1,2022-07-27 23:59:25,2022-07-27


### 3.2用户留存数据详情

In [5]:
def retained_sum(data4):
    data4['时间']= pd.to_datetime(data4['时间'])
    data4['created_at']= pd.to_datetime(data4['created_at'])
    data4['日期']= pd.to_datetime(data4['日期'])
    data4['time']= pd.to_datetime(data4['time'])

    data4['相隔时间'] = data4['created_at'] - data4['时间']

    data4['相隔时间'] = data4['相隔时间'].map(lambda x: x/np.timedelta64(1,'h'))

    data4['留存'] = np.where(((data4['相隔时间']>= 0.05  )   & (data4['相隔时间'] <24 )),'次日留存',np.where(((data4['相隔时间']>= 24  )   & (data4['相隔时间'] <186 )),'7日留存',np.where(((data4['相隔时间']>= 186  )   & (data4['相隔时间'] <360 )),'15日留存',np.where(((data4['相隔时间']>= 360 )  & (data4['相隔时间'] <720 )),'30日留存','未知'))))
    return data4

In [6]:
user_retained_1 = retained_sum(user_retained_1)
user_retained_2 = retained_sum(user_retained_2)

In [7]:
user_retained_1

,id,user_province,日期,时间,age,gender,group_id,created_at,time,相隔时间,留存
0,2140809,None,2022-07-01,2022-07-01 00:04:08,0,0,1,2022-07-01 00:04:08,2022-07-01,0.000000,未知
1,2140810,None,2022-07-01,2022-07-01 00:04:13,0,0,1,2022-07-01 00:04:13,2022-07-01,0.000000,未知
2,2140811,广西壮族自治区,2022-07-01,2022-07-01 00:12:57,36,2,2,2022-07-01 00:12:57,2022-07-01,0.000000,未知
3,2140811,广西壮族自治区,2022-07-01,2022-07-01 00:12:57,36,2,2,2022-07-01 00:13:29,2022-07-01,0.008889,未知
4,2140811,广西壮族自治区,2022-07-01,2022-07-01 00:12:57,36,2,2,2022-07-01 00:13:49,2022-07-01,0.014444,未知
...,...,...,...,...,...,...,...,...,...,...,...
68074,2191200,None,2022-07-25,2022-07-25 16:12:56,0,0,2,2022-07-25 17:54:37,2022-07-25,1.694722,次日留存
68075,2191856,None,2022-07-25,2022-07-25 17:54:45,0,0,1,2022-07-25 17:54:45,2022-07-25,0.000000,未知
68076,2191831,None,2022-07-25,2022-07-25 17:50:56,0,0,1,2022-07-25 17:54:51,2022-07-25,0.065278,次日留存
68077,2191857,None,2022-07-25,2022-07-25 17:54:55,0,0,1,2022-07-25 17:54:56,2022-07-25,0.000278,未知


In [8]:
user_retained_1_01 = user_retained_1[user_retained_1['留存'] == '次日留存']
user_retained_2_01 = user_retained_2[user_retained_2['留存'] == '次日留存']

In [9]:
print('本月用户次日留存复登录人数 ' + str(user_retained_1_01.drop_duplicates(subset='id',inplace=False)['id'].count()))
print('上月用户次日留存复登录人数 ' + str(user_retained_2_01.drop_duplicates(subset='id',inplace=False)['id'].count()))
print('用户次日留存复登录人数环比 ' + str((user_retained_1_01.drop_duplicates(subset='id',inplace=False)['id'].count() - user_retained_2_01.drop_duplicates(subset='id',inplace=False)['id'].count()) / user_retained_2_01.drop_duplicates(subset='id',inplace=False)['id'].count()))
print('本月用户次日留存率 ' + str(user_retained_1_01.drop_duplicates(subset='id',inplace=False)['id'].count() / joinuser_1.drop_duplicates(subset='id',inplace=False)['id'].count()))
print('上月用户次日留存率 ' + str(user_retained_2_01.drop_duplicates(subset='id',inplace=False)['id'].count() / joinuser_2.drop_duplicates(subset='id',inplace=False)['id'].count()))

本月用户次日留存复登录人数 2902
上月用户次日留存复登录人数 3083
用户次日留存复登录人数环比 -0.0587090496269867


NameError: name 'joinuser_1' is not defined

In [ ]:
print(user_retained_1[(user_retained_1['相隔时间'] > 1 ) &(user_retained_1['相隔时间'] < 24 )  ].drop_duplicates(subset='id',inplace=False)['id'].count() )
print(user_retained_1[(user_retained_1['相隔时间'] > 24 ) &(user_retained_1['相隔时间'] < 48 )  ].drop_duplicates(subset='id',inplace=False)['id'].count() )
print(user_retained_1[(user_retained_1['相隔时间'] > 48 ) &(user_retained_1['相隔时间'] < 72 )  ].drop_duplicates(subset='id',inplace=False)['id'].count() )
print(user_retained_1[(user_retained_1['相隔时间'] > 72 ) &(user_retained_1['相隔时间'] < 96 )  ].drop_duplicates(subset='id',inplace=False)['id'].count() )
print(user_retained_1[(user_retained_1['相隔时间'] > 96 ) &(user_retained_1['相隔时间'] < 120 )  ].drop_duplicates(subset='id',inplace=False)['id'].count() )
print(user_retained_1[(user_retained_1['相隔时间'] > 120 ) &(user_retained_1['相隔时间'] < 144 )  ].drop_duplicates(subset='id',inplace=False)['id'].count() )
print(user_retained_1[(user_retained_1['相隔时间'] > 144 ) &(user_retained_1['相隔时间'] < 168 )  ].drop_duplicates(subset='id',inplace=False)['id'].count() )

In [ ]:
print(user_retained_2[(user_retained_2['相隔时间'] > 1 ) &(user_retained_2['相隔时间'] < 24 )  ].drop_duplicates(subset='id',inplace=False)['id'].count() )
print(user_retained_2[(user_retained_2['相隔时间'] > 24 ) &(user_retained_2['相隔时间'] < 48 )  ].drop_duplicates(subset='id',inplace=False)['id'].count() )
print(user_retained_2[(user_retained_2['相隔时间'] > 48 ) &(user_retained_2['相隔时间'] < 72 )  ].drop_duplicates(subset='id',inplace=False)['id'].count() )
print(user_retained_2[(user_retained_2['相隔时间'] > 72 ) &(user_retained_2['相隔时间'] < 96 )  ].drop_duplicates(subset='id',inplace=False)['id'].count() )
print(user_retained_2[(user_retained_2['相隔时间'] > 96 ) &(user_retained_2['相隔时间'] < 120 )  ].drop_duplicates(subset='id',inplace=False)['id'].count() )
print(user_retained_2[(user_retained_2['相隔时间'] > 120 ) &(user_retained_2['相隔时间'] < 144 )  ].drop_duplicates(subset='id',inplace=False)['id'].count() )
print(user_retained_2[(user_retained_2['相隔时间'] > 144 ) &(user_retained_2['相隔时间'] < 168 )  ].drop_duplicates(subset='id',inplace=False)['id'].count() )

In [ ]:
user_retained_1
retained_user = {}
for i in time_list1:
    x = user_retained_1_01[user_retained_1_01['time'] == i]
    y = x.drop_duplicates(subset='id',inplace = False)
    renshu = y['id'].count()
    retained_user[i] = renshu
retained_user = pd.DataFrame.from_dict(retained_user, orient='index',columns=['留存人数'])

In [ ]:
retained_user